In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Going through the input file
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

'''We could also have used XLM-Roberta model instead of Distilbert model which have given a better score but was
   computationaly very heavy with about three times more parameter than distilbert to be trained during training.'''  

In [3]:
# Function for encoding the text
def fast_encode(texts, tokenizer, batch_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    # Batchwise Encoding
    for i in tqdm(range(0, len(texts), batch_size)):
        text_chunk = texts[i:i+batch_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

'''In case of XLM Roberta we may use the following 
    def regular_encode(texts, tokenizer, maxlen=512):
        encoded = tokenizer.batch_encode_plus(texts, return_attention_masks=False, return_token_type_ids=False,pad_to_max_length=True,max_length=maxlen)
        return np.array(encoded['input_ids'])'''

In [4]:
#function to create a model with pretrained bert layers
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(264, activation = 'relu')(cls_token)
    
    out = Dense(1, activation='sigmoid')(out)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [6]:
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync # Tried 32,64 inplace of 16 but this gave the best result
MAX_LEN = 192

In [7]:
BATCH_SIZE

128

In [8]:
# The real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

'''In case of XLM Roberta model's tokenizer we would use AutoTokenizer instead of BertWordPieceTokenizer like
    tokenizer = AutoTokenizer.from_pretrained('jplu/tf-xlm-roberta-large')'''

Tokenizer(vocabulary_size=119547, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False, wordpieces_prefix=##)

In [9]:
# Loading Data
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

# Combining train1 with subsets of train2 having label as toxic 
train = pd.concat([train1[['comment_text', 'toxic']],train2[['comment_text', 'toxic']].query('toxic==1'),
                train2[['comment_text', 'toxic']].query('toxic==0').sample(n=150000, random_state=0)])

In [10]:
# Encode the data
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)
'''In case of XLM Roberta
    x_train = regular_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
    x_valid = regular_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
    x_test = regular_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)'''

y_train = train.toxic.values
y_valid = valid.toxic.values

In [11]:
#DataSet object
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

valid_dataset = (tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(BATCH_SIZE).cache().prefetch(AUTO))

test_dataset = (tf.data.Dataset.from_tensor_slices(x_test).batch(BATCH_SIZE))

In [12]:
# Loading the model on tpu

with strategy.scope():
    transformer_layer = (transformers.TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased'))
    '''For XLM Roberta we will use
        transformer_layer = transformers.TFAutoModel.from_pretrained('jplu/tf-xlm-roberta-large')'''
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 264)               203016    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 265       
Total params: 134,937,361
Trainable params: 134,937,361
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Training the model
n_steps = x_train.shape[0]//BATCH_SIZE # Batch size 
train_history = model.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=EPOCHS)

# Did few epochs on validation set as it has some examples with languages other than english,boosting its performance
n_steps = x_valid.shape[0]//BATCH_SIZE #Batch size
train_history_2 = model.fit(valid_dataset.repeat(),steps_per_epoch=n_steps,epochs=EPOCHS*2)

Train for 3795 steps, validate for 63 steps
Epoch 1/3
3795/3795 [==============================] - 471s 124ms/step - loss: 0.0668 - accuracy: 0.9740 - val_loss: 0.3542 - val_accuracy: 0.8464
Epoch 2/3
3795/3795 [==============================] - 413s 109ms/step - loss: 0.0501 - accuracy: 0.9802 - val_loss: 0.4273 - val_accuracy: 0.8459
Epoch 3/3
3795/3795 [==============================] - 416s 109ms/step - loss: 0.0428 - accuracy: 0.9828 - val_loss: 0.4490 - val_accuracy: 0.8473
Train for 62 steps
Epoch 1/6
62/62 [==============================] - 17s 276ms/step - loss: 0.3165 - accuracy: 0.8652
Epoch 2/6
62/62 [==============================] - 25s 403ms/step - loss: 0.2136 - accuracy: 0.9098
Epoch 3/6
62/62 [==============================] - 7s 108ms/step - loss: 0.1374 - accuracy: 0.9431
Epoch 4/6
62/62 [==============================] - 7s 109ms/step - loss: 0.0807 - accuracy: 0.9679
Epoch 5/6
62/62 [==============================] - 7s 108ms/step - loss: 0.0853 - accuracy: 0.9651

In [14]:
# Saving in the submission file
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

499/499 [==============================] - 39s 77ms/step


In [ ]:
# The AUROC Score for this is 0.8739 according to kaggle on its testing data
# My username is DIVYANSH SRIVASTAVA and my current position in the contest was 562. 
#Thank You